<a href="https://colab.research.google.com/github/KartikayBhardwaj-dev/Ai_lab/blob/main/ai_lab_week07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RAG**
1. Retrieval Augmented Generation
2. LLm trained on specific data so if its trained on september it answers you till september not recent
3. solution is :
- Retarain or Finetuning llm
- Use Prompt Stuffing, pass event with query with prompt, giving recent info to llm in prompt
- RAG:
4. RAG
- Retrieval: get some info using some source
- Generation:  info from retriver used to generate
5. uses of RAG
- Somethings keeps changing rag can be used
- info not available online like medical details or legal details rag can use our details and make some comment
6. Workflow of RAG
- user asks a query -> vector databases will access -> relevant documents will retrieve for users query ->
prompt added to user prompt -> feed to llm (prompt) -> llm produce output
7. Prepare data for Retreival,
- load pdf or .txt .html ,
- we split the each document into small chunks
- embeddings (convert chunk to numerical)
- store vectors we get from embedding we call it as vector stores
8. Chunk overlap
- Chunk1 ends with A france and is
- chunk2 starts with A france and is
- they share something common we put that on chunk overlap layer this usualling keep 10 20 % percent data
9. Rag vs Finetuning
10. Approach
- RAG: add external knowledge to the model at query time
- Finetuning: changes model weights to learn from your data
11. Data needed
- RAG: you can pass unstructured documents
- Finetuning: you need to preprocessed data
12. Use case type
- RAG: when the knowledge keeps changing
- Finetuning: when you need to teach a new skill to llm which will stick


#RAG Implementation

#Installing

In [1]:
!pip install -U langchain langchain-groq langchain-core langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [20]:
import langchain
import langchain_core
import langchain_community
import os
from google.colab import userdata
import langchain_groq
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

print("langchain version:", langchain.__version__)
print("langchain-core version:", langchain_core.__version__)
print("langchain-community version:", langchain_community.__version__)
# print("langchain-groq version:", langchain_groq.__version__)

langchain version: 1.1.0
langchain-core version: 1.1.0
langchain-community version: 0.4.1


Testing

In [21]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",  # ✔ new working model
    groq_api_key=os.environ["GROQ_API_KEY"],
    temperature=0.7
)

llm.invoke("capital of kartikay?").content

'I couldn\'t find any information about a place named "Kartikay" that has a known capital. It is possible that Kartikay is a misspelling or a less-known location. If you meant "Kashmir," the capital is Srinagar.'

So the answer where it cant find input given is not where chat groq model is trained on, here comes rag

I just add a rag_file.txt where i give some info about kartikay

#RAG steps
1. Read data for retrieval
2. split the text
3. produce embeddings for splits
4. store embeddings in vectorDB
5. Create retreiver to retrieve the embeddings from the vectorDB
6. combine the llm and the retriever and produce results

Read through pdf

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 6.5 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Test.pdf")
docs = loader.load()

In [4]:
for doc in docs:
  print(doc.page_content)
  print(doc.metadata)

Kartikay is a country in south eastern part of earth
It has 29 states and 2 union territories 
Capital of kartikay is andaman
President of kartikay is India
{'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251126082445Z00'00'", 'moddate': "D:20251126082445Z00'00'", 'source': 'Test.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


#Split the text

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=15)

In [6]:
chunks = splitter.split_documents(docs)

In [7]:
chunks

[Document(metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251126082445Z00'00'", 'moddate': "D:20251126082445Z00'00'", 'source': 'Test.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Kartikay is a country in south eastern part of'),
 Document(metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251126082445Z00'00'", 'moddate': "D:20251126082445Z00'00'", 'source': 'Test.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='part of earth'),
 Document(metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251126082445Z00'00'", 'moddate': "D:20251126082445Z00'00'", 'source': 'Test.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='It has 29 states and 2 union territories'),
 Document(metadata={'producer': 'macOS Version 15.6.1 (Build 24G90)

In [8]:
for chunk in chunks:
  print(chunk.page_content)
  print("----")

Kartikay is a country in south eastern part of
----
part of earth
----
It has 29 states and 2 union territories
----
Capital of kartikay is andaman
----
President of kartikay is India
----


In [9]:
texts = [doc.page_content for doc in chunks]
texts

['Kartikay is a country in south eastern part of',
 'part of earth',
 'It has 29 states and 2 union territories',
 'Capital of kartikay is andaman',
 'President of kartikay is India']

#produce embeddings for splits

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en"
)

vectors = embedding_model.embed_documents(texts)

/tmp/ipython-input-13013402.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
len(vectors)

5

#Store embedding into VectorDB
- we will use FAISS, facebook ai similarity search

In [12]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 72.1 MB/s eta 0:00:00


In [13]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(chunks, embedding_model)

#some query checking

In [15]:
retrive_doc = vectorstore.similarity_search("How many states country have?", k=1)
retrive_doc

[Document(id='2d75e373-d20c-4303-99c2-7862f9058a50', metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251126082445Z00'00'", 'moddate': "D:20251126082445Z00'00'", 'source': 'Test.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='It has 29 states and 2 union territories')]

#Create retriever

In [16]:
retriever = vectorstore.as_retriever(search_kwargs={"k":1})

#Testing

In [17]:
docs = retriever.invoke("who is president of kartikay?")
for d in docs:
  print(d.page_content)
  print(d.metadata)


President of kartikay is India
{'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20251126082445Z00'00'", 'moddate': "D:20251126082445Z00'00'", 'source': 'Test.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


#Combine llm and Retriever and produce results


In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#defining prompt template
template = """Use the following context to answer the question
Use three sentences maximum and keep the answer concise

Context: {context}
Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

#create ragchain
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

#RAG pipeline
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#query
query1 = "what is kartikay?"
response1 = rag_chain.invoke(query1)
print(response1)


Unfortunately, I couldn't find any information about a country called "Kartikay" in the southeastern part of the world. It's possible that it's a fictional or lesser-known location. If you have more information, I'd be happy to help.


In [26]:
query2 = "president of Kartikay?"
response2 = rag_chain.invoke(query2)
print(response2)

The President of Kartikay is India.
